## 利用TensorBoard对线性回归模型训练过程绘制loss曲线
SummaryWriter类，用来把训练过程中的数据（比如损失、准确率、模型图、图像、直方图等）写入日志文件，通过Tensor Board可视化。

重新训练模型，指定的记录位置处会产生文件

在Anaconda Prompt里，激活环境，运行：

tensorboard --logdir="D:/PDXAC/code/deep_learning/runs/"

In [ ]:
import torch
# 导入SummaryWriter类
from torch.utils.tensorboard import SummaryWriter

# 确保CUDA可用
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")  # 如果没有GPU，改成"cpu"

# 生成数据
inputs = torch.rand(100,3) # 100条数据，每条3个feature
weights = torch.tensor([[1.1],[2.2],[3.3]]) #预设的权重
bias = torch.tensor(4.4)      #预设的偏置
targets = inputs @ weights + bias +0.1*torch.randn(100,1)  #x @ w + b + 噪声

# 创建一个SummaryWriter实例，通过log_dir设置记录文件的位置
writer = SummaryWriter(log_dir = "D:/PDXAC/code/deep_learning/runs/")

# 初始化参数时直接放在CUDA上，启用梯度计算
w = torch.rand((3,1), requires_grad=True, device=device) #权重
b = torch.rand((1,), requires_grad=True, device=device)    #偏置

# 将数据移至相同设备
inputs = inputs.to(device)
targets = targets.to(device)

# 设置超参数
epoch = 10000 # 迭代次数
lr = 0.003    # 学习率

for i in range(epoch):
    outputs = inputs @ w + b  # 前向传播，计算预测值
    loss = torch.mean(torch.square(outputs - targets)) # 计算均方误差MSE损失函数
    print("loss:",loss.item())  # .item()把只含一个数的张量，转换成普通的Python数字标量
    
    # 训练的每一步写入loss值，传入三个参数分别为：标签名，loss值，训练的步数
    writer.add_scalar("loss/train",loss.item(),i)
    loss.backward()  # 反向传播，计算梯度，存储到 w.grad, b.grad
    
    with torch.no_grad():  # 下面的计算不需要用梯度，关闭节省内存
        w -= lr * w.grad  # 更新权重
        b -= lr * b.grad  # 更新偏置
        
    # 清零梯度，避免梯度值一直累积
    w.grad.zero_()
    b.grad.zero_()
    
print("训练后的权重w:", w)
print("训练后的偏置b:", b)

loss: 46.21493148803711
loss: 45.20921325683594
loss: 44.225521087646484
loss: 43.26335525512695
loss: 42.3222541809082
loss: 41.40175247192383
loss: 40.501407623291016
loss: 39.62076187133789
loss: 38.759395599365234
loss: 37.91688919067383
loss: 37.09281921386719
loss: 36.28678894042969
loss: 35.4984016418457
loss: 34.72727584838867
loss: 33.973026275634766
loss: 33.235286712646484
loss: 32.513694763183594
loss: 31.80789566040039
loss: 31.1175479888916
loss: 30.442312240600586
loss: 29.781850814819336
loss: 29.135848999023438
loss: 28.503984451293945
loss: 27.88595199584961
loss: 27.281444549560547
loss: 26.690170288085938
loss: 26.111833572387695
loss: 25.546157836914062
loss: 24.99285888671875
loss: 24.451671600341797
loss: 23.922334671020508
loss: 23.404573440551758
loss: 22.898149490356445
loss: 22.40280532836914
loss: 21.91830062866211
loss: 21.44440460205078
loss: 20.980873107910156
loss: 20.527490615844727
loss: 20.084028244018555
loss: 19.650270462036133
loss: 19.226005554199

![image.png](attachment:image.png)

这是一个非常理想的loss曲线，前边一段训练过程loss快速下降，到后边loss逐渐变化很小，证明已经收敛。整个训练过程loss变化也很平稳，因为整个曲线很平滑。

根据loss曲线，可以判断在loss几乎不变，看起来是一条平行于x轴的直线时，我们认为模型收敛，梯度下降已经不能进一步减少损失了，此时可以终止训练，得到最优模型。

[CSV](../deep_learning/runs/csv.csv)  是TensorBoard事件日志（event）导出的CSV，记录写入的标量数据

Wall time 事件发生的真实时间

Step 训练的迭代次数

Value 事件对应的值(loss值)
